# Consolidação de Base de Dados - Web of Science (Plain Text)
Diferente da IEEE e Scopus, a Web of Science exporta os dados em formato de texto estruturado por tags. 
Cada publicação inicia na tag de tipo de documento (`PT J`, por exemplo) e termina com a tag de fim de registro (`ER`).

In [1]:
import os

pasta_origem_wos = "/home/silvio/Projetos/TEMAC/WOS"

if not os.path.exists(pasta_origem_wos):
    print(f"Atenção: O diretório {pasta_origem_wos} não foi encontrado.")
else:
    arquivos_txt_wos = [f for f in os.listdir(pasta_origem_wos) if f.endswith('.txt')]
    print(f"Encontrados {len(arquivos_txt_wos)} arquivos .txt da WoS.")

Encontrados 9 arquivos .txt da WoS.


### 1. Função Analisadora (Parser) da WoS
O código abaixo lê as linhas do arquivo e agrupa cada registro em um bloco.
Ao mesmo tempo, ele mapeia as tags principais (como **TI** para Título, **DI** para DOI e **UT** para o ID único da Clarivate) para podermos usar como chave na hora de remover as duplicatas.

In [2]:
def parse_wos_file(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8-sig') as file:
        linhas = file.readlines()

    header_global = []
    registros = []
    
    bloco_raw_atual = []
    metadados_atuais = {}
    tag_atual = ""
    lendo_header = True

    for linha in linhas:
        # Remove a quebra de linha do final, mas mantém os espaços do início
        linha_limpa = linha.rstrip('\n') 
        
        # Ignora linhas 100% vazias entre os registros
        if not linha_limpa.strip():
            continue

        # Captura o cabeçalho inicial (FN e VR)
        if linha_limpa.startswith('FN ') or linha_limpa.startswith('VR '):
            if lendo_header:
                header_global.append(linha_limpa)
            continue

        # Quando encontra "PT " (Publication Type), começa um novo registro
        if linha_limpa.startswith('PT '):
            lendo_header = False

        if lendo_header:
            continue
            
        # Adiciona a linha ao bloco de texto bruto que será salvo no final
        bloco_raw_atual.append(linha_limpa)

        # Fim do registro
        if linha_limpa == 'ER':
            registros.append({
                'bloco_texto': '\n'.join(bloco_raw_atual),
                # Salvamos as chaves principais em minúsculo e sem quebras para facilitar a comparação
                'DI': metadados_atuais.get('DI', '').strip().lower(), # DOI
                'TI': metadados_atuais.get('TI', '').strip().lower(), # Título
                'UT': metadados_atuais.get('UT', '').strip().lower()  # ID Único WoS
            })
            bloco_raw_atual = []
            metadados_atuais = {}
            tag_atual = ""
            continue

        # Lógica para extrair as Tags e os Valores
        # Se a linha não começa com espaço e tem uma tag de 2 letras (Ex: "TI ")
        if not linha_limpa.startswith(' ') and len(linha_limpa) >= 3 and linha_limpa[2] == ' ':
            tag_atual = linha_limpa[0:2]
            valor = linha_limpa[3:]
            metadados_atuais[tag_atual] = valor
        elif tag_atual:
            # É uma linha de continuação (Ex: segunda linha de um abstract longo)
            metadados_atuais[tag_atual] += " " + linha_limpa.strip()

    return header_global, registros

print("Função de parser carregada com sucesso.")

Função de parser carregada com sucesso.


### 2. Etapa de Debug: Analisando as Tags Extraídas
Vamos ler todos os arquivos, processar usando nossa função e imprimir um exemplo do primeiro registro extraído para validar se as tags (TI, DI, UT) foram mapeadas corretamente antes da consolidação.

In [3]:
todos_os_registros = []
cabecalho_oficial = []

for arquivo in arquivos_txt_wos:
    caminho = os.path.join(pasta_origem_wos, arquivo)
    header, records = parse_wos_file(caminho)
    
    if not cabecalho_oficial and header:
        cabecalho_oficial = header # Salva o cabeçalho do primeiro arquivo
        
    todos_os_registros.extend(records)
    print(f"Lido: {arquivo} - {len(records)} publicações encontradas.")

print("\n--- DEBUG: Exemplo das chaves de indexação do PRIMEIRO registro ---")
if todos_os_registros:
    exemplo = todos_os_registros[0]
    print(f"Título (TI): {exemplo['TI'][:80]}...") # Mostra os primeiros 80 caracteres
    print(f"DOI (DI)   : {exemplo['DI']}")
    print(f"WoS ID (UT): {exemplo['UT']}")

Lido: savedrecs.txt - 11 publicações encontradas.
Lido: savedrecs (5).txt - 10 publicações encontradas.
Lido: savedrecs (3).txt - 84 publicações encontradas.
Lido: savedrecs (2).txt - 42 publicações encontradas.
Lido: savedrecs (4).txt - 654 publicações encontradas.
Lido: savedrecs (8).txt - 9 publicações encontradas.
Lido: savedrecs (6).txt - 43 publicações encontradas.
Lido: savedrecs (7).txt - 20 publicações encontradas.
Lido: savedrecs (1).txt - 623 publicações encontradas.

--- DEBUG: Exemplo das chaves de indexação do PRIMEIRO registro ---
Título (TI): review of open-source software for developing heterogeneous data management syst...
DOI (DI)   : 10.1093/bioadv/vbaf168
WoS ID (UT): wos:001543196800001


### 3. Remoção de Duplicatas e Consolidação
Como não estamos usando o Pandas aqui, usaremos um dicionário nativo do Python. Ao usar a chave única do artigo (UT, DOI ou Título) como chave do dicionário, publicações repetidas irão sobrescrever a anterior, mantendo a base final limpa.

In [4]:
# Dicionário para guardar registros únicos
registros_deduplicados = {}

for reg in todos_os_registros:
    # A prioridade de chave única: 1º WoS ID (UT), 2º DOI (DI), 3º Título (TI)
    chave_unica = reg['UT']
    
    if not chave_unica:
        chave_unica = reg['DI']
        
    if not chave_unica:
        chave_unica = reg['TI']
    
    # Adiciona ao dicionário. Se a chave já existir, ele simplesmente sobrescreve (remove a duplicata)
    if chave_unica:
        registros_deduplicados[chave_unica] = reg['bloco_texto']

total_original = len(todos_os_registros)
total_final = len(registros_deduplicados)

print("--- Resumo da Limpeza (WoS) ---")
print(f"Registros brutos lidos: {total_original}")
print(f"Registros únicos mantidos: {total_final}")
print(f"Duplicatas removidas: {total_original - total_final}")

--- Resumo da Limpeza (WoS) ---
Registros brutos lidos: 1496
Registros únicos mantidos: 1290
Duplicatas removidas: 206


### 4. Geração do Arquivo Final em TXT
Reconstruindo o arquivo seguindo exatamente o padrão original exigido por ferramentas de bibliometria ou importadores (como Mendeley/Zotero).

In [5]:
arquivo_saida_wos = "/home/silvio/Projetos/TEMAC/ajustes_BD/base_wos_consolidada.txt"

with open(arquivo_saida_wos, 'w', encoding='utf-8') as f:
    # 1. Escreve o cabeçalho "FN ..." e "VR ..."
    for linha_header in cabecalho_oficial:
        f.write(linha_header + "\n")
    
    # 2. Escreve os registros únicos
    for bloco_raw in registros_deduplicados.values():
        f.write(bloco_raw + "\n\n") # Adiciona duas quebras para garantir a linha em branco entre registros

print(f"\nArquivo final padronizado salvo com sucesso: {arquivo_saida_wos}")


Arquivo final padronizado salvo com sucesso: /home/silvio/Projetos/TEMAC/ajustes_BD/base_wos_consolidada.txt
